In [ ]:
!pip install wandb -qU

In [ ]:
import wandb
wandb.login()

In [ ]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import re

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)
logger.handlers[0] = c_handler

In [ ]:
sweep_config = {
    'method': 'random'
    }

metric ={
      'name': 'val_accuracy',
      'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'activation':
    {
        'values' : ['relu']   
    },
    'camadas':
    {
        'values':[1,2,3]
    },
    'fc_layer_size': {
         'distribution': 'q_log_uniform_values',
          'q': 8,
          'min':10,
          'max':400,
        },

    }

sweep_config['parameters'] = parameters_dict



parameters_dict.update({
    'learning_rate': {
        'values':['constant','adaptive']
      },
      'learning_rate_init': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
 
    })

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='classifier_mlp')

In [ ]:
args = {
  "project_name": "classifier_mlp",
  "train_feature_artifact": "train_x:latest",
  "train_target_artifact": "train_y:latest",
  "jobs": -1,
  "encoder": "target_encoder",
  "inference_model": "model"
}

In [ ]:
qtd =1

In [ ]:
def train(config=None):
  global qtd
    
  with wandb.init(entity="igordias",project=args["project_name"], job_type="Train") as run:

    wandb.run.name = 'treino' + str(qtd)
    qtd+=1
    config = wandb.config

    logger.info("Downloading the training data")
    train_x_artifact = run.use_artifact(args["train_feature_artifact"])
    train_x_path = train_x_artifact.file()
    train_y_artifact = run.use_artifact(args["train_target_artifact"])
    train_y_path = train_y_artifact.file()

    # unpacking the artifacts
    train_x = joblib.load(train_x_path)
    train_y = joblib.load(train_y_path)
    le = LabelEncoder()
    train_y = le.fit_transform(train_y)

    # train a MLP classifier on the raw pixel intensities
    logger.info("[INFO] training MLP classifier...")
    model=""
    if(config.camadas==1):
      model = MLPClassifier(hidden_layer_sizes=config.fc_layer_size,learning_rate=config.learning_rate,max_iter=1000,learning_rate_init=config.learning_rate_init)
    elif(config.camadas==2):
      model = MLPClassifier(hidden_layer_sizes=(config.fc_layer_size,config.fc_layer_size),learning_rate=config.learning_rate,max_iter=1000,learning_rate_init=config.learning_rate_init)
    else:
      model = MLPClassifier(hidden_layer_sizes=(config.fc_layer_size,config.fc_layer_size,config.fc_layer_size),learning_rate=config.learning_rate,max_iter=1000,learning_rate_init=config.learning_rate_init)

    model.fit(train_x, train_y)

    test_x_artifact = run.use_artifact("test_x:latest")
    test_x_path = test_x_artifact.file()
    test_y_artifact = run.use_artifact("test_y:latest")
    test_y_path = test_y_artifact.file()

    # unpacking the artifacts
    test_x = joblib.load(test_x_path)
    test_y = joblib.load(test_y_path)


    test_y = le.fit_transform(test_y)

    logger.info("[INFO] evaluating MLP classifier...")
    predict = model.predict(test_x)
    score = model.score(test_x,test_y)
    
    logger.info("Test Evaluation metrics")

    fbeta = fbeta_score(test_y, predict, beta=1, zero_division=1,average='weighted')
    precision = precision_score(test_y, predict, zero_division=1,average='weighted')
    recall = recall_score(test_y, predict, zero_division=1,average='weighted')
    acc = accuracy_score(test_y, predict)

    logger.info("Test Accuracy: {}".format(acc))
    logger.info("Test Precision: {}".format(precision))
    logger.info("Test Recall: {}".format(recall))
    logger.info("Test F1: {}".format(fbeta))

    wandb.log({"val_accuracy":acc,"Precision":precision,"score": score,'loss':model.best_loss_})    

    logger.info(classification_report(test_y, predict,target_names=le.classes_))


In [ ]:
wandb.agent(sweep_id, train, count=300)